In [1]:
from utils import extract_data, load_data
import os
import pandas as pd
from IPython.display import display

In [2]:
def extract(model: str):
    raw_data = load_data(os.path.join('data', 'raw', f'bonbanh.{model}.pkl'))
    clean_data = []
    for e in raw_data:
        clean_data.append(extract_data(e))
    df = pd.DataFrame(clean_data)
    display(df.head())
    df.to_pickle(os.path.join('data', 'clean', f'bonbanh.{model}.pkl'))

In [3]:
# extract('audi')
# extract('bentley')
# extract('bmw')
# extract('chevrolet')
# extract('daewoo')
# extract('ford')
# extract('honda')
# extract('hyundai')
# extract('isuzu')
# extract('jeep')
# extract('kia')
# extract('landrover')
# extract('lexus')
# extract('mazda')
# extract('mercedes_benz')
# extract('mg')
# extract('mini')
# extract('mitsubishi')
# extract('nissan')
# extract('peugeot')
# extract('porsche')
# extract('subaru')
# extract('suzuki')
# extract('toyota')
# extract('vinfast')
# extract('volkswagen')
# extract('volvo')

In [4]:
import pickle

def combine_pickle_files(directory_path, output_file):
    # Kiểm tra xem file data.pkl đã tồn tại chưa
    if os.path.exists(output_file):
        with open(output_file, 'rb') as f:
            data_df = pickle.load(f)  # Nếu có, đọc dữ liệu hiện tại từ file
    else:
        data_df = pd.DataFrame()  # Nếu không, khởi tạo một DataFrame trống

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.pkl'):
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'rb') as f:
                content = pickle.load(f)
                if isinstance(content, pd.DataFrame):
                    data_df = pd.concat([data_df, content], ignore_index=True)

    with open(output_file, 'wb') as out:
        pickle.dump(data_df, out, protocol=pickle.HIGHEST_PROTOCOL)
        print(len(data_df))

# Sử dụng hàm:
directory_path = 'data/clean'
output_file = 'data/clean/data.pkl'
combine_pickle_files(directory_path, output_file)

30881


In [5]:
import os
import pickle
import pandas as pd
# Đường dẫn đến thư mục chứa các file .pkl
folder_path = 'data/clean'

# Duyệt qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    # Kiểm tra nếu file có phần mở rộng là .pkl
    if filename.endswith('.pkl'):
        file_path = os.path.join(folder_path, filename)
        
        # Mở và đọc file .pkl
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        
        # In ra độ dài của dữ liệu
        print(f'Độ dài của file {filename} là: {len(data)}')

Độ dài của file bonbanh.nissan.pkl là: 300
Độ dài của file bonbanh.mg.pkl là: 376
Độ dài của file bonbanh.ford.pkl là: 3155
Độ dài của file bonbanh.mitsubishi.pkl là: 1474
Độ dài của file data.pkl là: 30881
Độ dài của file bonbanh.vinfast.pkl là: 969
Độ dài của file bonbanh.mercedes_benz.pkl là: 3329
Độ dài của file bonbanh.bmw.pkl là: 682
Độ dài của file bonbanh.hyundai.pkl là: 3475
Độ dài của file bonbanh.honda.pkl là: 1352
Độ dài của file bonbanh.toyota.pkl là: 5860
Độ dài của file bonbanh.chevrolet.pkl là: 554
Độ dài của file bonbanh.porsche.pkl là: 429
Độ dài của file bonbanh.landrover.pkl là: 411
Độ dài của file bonbanh.bentley.pkl là: 65
Độ dài của file bonbanh.jeep.pkl là: 89
Độ dài của file bonbanh.volvo.pkl là: 150
Độ dài của file bonbanh.mazda.pkl là: 2147
Độ dài của file bonbanh.audi.pkl là: 315
Độ dài của file bonbanh.peugeot.pkl là: 275
Độ dài của file bonbanh.isuzu.pkl là: 120
Độ dài của file bonbanh.lexus.pkl là: 1177
Độ dài của file bonbanh.volkswagen.pkl là: 361
Độ dà